In [ ]:
install_packages=False
if install_packages:
    !pip install scipy
    !pip install numpy
    !pip install matlab
    !pip install matlabengine
    !pip install openpyxl

In [6]:
import matlab.engine
import numpy as np
import scipy.io
import openpyxl

## Simulation test

In [13]:
# MATLAB 엔진 시작
eng = matlab.engine.start_matlab()

# Simulink 모델 실행 및 결과 값 받기
def run_simulink_model(action):#, agent):
    # MATLAB에서 변수 설정
    eng.workspace['action'] = matlab.double(action)#.tolist())
    #eng.workspace['agent'] = agent
    eng.eval("noiseStd=0.01", nargout=0)
    eng.eval("load trainedAgent_3D_06_22_2024_1810.mat", nargout=0)
    # Simulink 모델 실행
    eng.eval("simout = sim('walkingRobotRL3D_pytest2.slx');", nargout=0)
    #print(eng.workspace.keys())
    # 결과 값 가져오기
    result=[]
    result.append(eng.workspace['cumulative_reward'])
    result.append(eng.workspace['yaw'])
    result.append(eng.workspace['pitch'])
    result.append(eng.workspace['roll'])
    result.append(eng.workspace['X'])
    result.append(eng.workspace['Y'])
    result.append(eng.workspace['Z'])
    result.append(eng.workspace['VX'])
    result.append(eng.workspace['VY'])
    result.append(eng.workspace['VZ'])
    result.append(eng.workspace['AX'])
    result.append(eng.workspace['AY'])
    result.append(eng.workspace['AZ'])
    result.append(eng.workspace['wx'])
    result.append(eng.workspace['wy'])
    result.append(eng.workspace['wz'])
    result.append(eng.workspace['actPower'])
    result.append(eng.workspace['R_ankle_roll_angle'])
    result.append(eng.workspace['R_ankle_roll_speed'])
    result.append(eng.workspace['R_ankle_roll_torque'])
    result.append(eng.workspace['R_ankle_pitch_angle'])
    result.append(eng.workspace['R_ankle_pitch_speed'])
    result.append(eng.workspace['R_ankle_pitch_torque'])
    result.append(eng.workspace['R_knee_angle'])
    result.append(eng.workspace['R_knee_speed'])
    result.append(eng.workspace['R_knee_torque'])
    result.append(eng.workspace['R_hip_roll_angle'])
    result.append(eng.workspace['R_hip_roll_speed'])
    result.append(eng.workspace['R_hip_roll_torque'])
    result.append(eng.workspace['R_hip_pitch_angle'])
    result.append(eng.workspace['R_hip_pitch_speed'])
    result.append(eng.workspace['R_hip_pitch_torque'])
    result.append(eng.workspace['L_ankle_roll_angle'])
    result.append(eng.workspace['L_ankle_roll_speed'])
    result.append(eng.workspace['L_ankle_roll_torque'])
    result.append(eng.workspace['L_ankle_pitch_angle'])
    result.append(eng.workspace['L_ankle_pitch_speed'])
    result.append(eng.workspace['L_ankle_pitch_torque'])
    result.append(eng.workspace['L_knee_angle'])
    result.append(eng.workspace['L_knee_speed'])
    result.append(eng.workspace['L_knee_torque'])
    result.append(eng.workspace['L_hip_roll_angle'])
    result.append(eng.workspace['L_hip_roll_speed'])
    result.append(eng.workspace['L_hip_roll_torque'])
    result.append(eng.workspace['L_hip_pitch_angle'])
    result.append(eng.workspace['L_hip_pitch_speed'])
    result.append(eng.workspace['L_hip_pitch_torque'])

    #sensor_res=eng.workspace['yaw']#고치기
    #reward_result = eng.workspace['simout1']
    #print(result)
    # 필요한 데이터를 추출하여 numpy 배열로 변환
    #state = np.array(result)
    
    return result

In [66]:
# Don't execute
# 에이전트 모델 불러오기
def load_agent_model(mat_file):
    # MATLAB 함수 호출
    mat_file_res = scipy.io.loadmat(mat_file)
    print(mat_file_res.keys())
    print(mat_file_res)
    agent=mat_file_res['None'][0]
    #agent = eng.load_agent_model(mat_file)
    return agent

In [65]:
# Don't execute
#mat_file = 'trainedAgent_3D_06_22_2024_1810.mat'
#mat_file_res = scipy.io.loadmat(mat_file)
#print(mat_file_res.keys())
#print(mat_file_res)
#print(mat_file_res['None'][0])

(b'agent', b'MCOS', b'rl.agent.rlDDPGAgent', array([[3707764736],
       [         2],
       [         1],
       [         1],
       [         1],
       [        23]], dtype=uint32))


In [14]:
# MAT 파일 경로
mat_file = 'trainedAgent_3D_06_22_2024_1810.mat'

# 에이전트 모델 불러오기
#agent = load_agent_model(mat_file)

# 초기 동작 예시
action = [0, 0]

# 시뮬레이션 실행 및 결과 값 출력
result = run_simulink_model(action)#, agent)
#print(sensor_res, reward_result)


In [9]:
len(result)

47

In [10]:
result[0]['time']#time
result[0]['signals']['values']#->np.array->datas
result[0]['signals']['label']#->feature name

'cumulative_reward'

In [15]:
label_list=[]
for i in range(len(result)):
    label_list.append(result[i]['signals']['label'])

## Save to Excel file test

In [22]:
import openpyxl
from openpyxl import Workbook

# 새로운 엑셀 워크북 생성
wb = Workbook()

# 기본 시트 이름 변경
ws = wb.active
ws.title = label_list[0]

# 추가 시트 생성
sheets = label_list[1:]
for sheet_name in sheets:
    wb.create_sheet(title=sheet_name)

data=[1,2,3,4,5]
sim_count=2
for sim_num in range(0,sim_count):
    # 초기 동작 예시
    action = [0, 0]
    # 시뮬레이션 실행
    result = run_simulink_model(action)

    # 각 시트에 열 추가
    for feature_num in len(feature_names):
        ws = wb[feature_names[feature_num]]
        # A1 셀에 데이터 추가 (예시)
        #ws['A1'] = [1,2,3,4,5]
        # 열에 데이터 추가 (예시)
        #for row in range(2, 11):
        #   ws[f'A{row}'] = f"Data {row-1}"
        data=np.array(result[feature_num]['signals']['values'])
        for i, value in enumerate(data, start=1):
            ws.cell(row=i, column=sim_num, value=value)
        #for i, value in enumerate(data, start=2):
           # ws.cell(row=i, column=2, value=value) #column을 1부터 100까지

# 엑셀 파일 저장
wb.save("example.xlsx")


## FINAL: Simulation and Save

여기부터 실행

In [1]:
import matlab.engine
import numpy as np
import scipy.io
import openpyxl
from openpyxl import Workbook
import datetime

matlab engine start & model run function

In [2]:
# MATLAB 엔진 시작
eng = matlab.engine.start_matlab()

# Simulink 모델 실행 및 결과 값 받기
def run_simulink_model(agent_name, noiseStd, action):#, agent):
    # MATLAB에서 변수 설정
    eng.workspace['action'] = matlab.double(action)#.tolist())
    #eng.workspace['agent'] = agent
    eng.eval("noiseStd="+str(noiseStd), nargout=0)
    eng.eval("load "+ agent_name, nargout=0)
    # Simulink 모델 실행
    eng.eval("simout = sim('walkingRobotRL3D_pytest2.slx');", nargout=0)

    # 결과 값 가져오기
    result=[]
    result.append(eng.workspace['cumulative_reward'])
    result.append(eng.workspace['yaw'])
    result.append(eng.workspace['pitch'])
    result.append(eng.workspace['roll'])
    result.append(eng.workspace['X'])
    result.append(eng.workspace['Y'])
    result.append(eng.workspace['Z'])
    result.append(eng.workspace['VX'])
    result.append(eng.workspace['VY'])
    result.append(eng.workspace['VZ'])
    result.append(eng.workspace['AX'])
    result.append(eng.workspace['AY'])
    result.append(eng.workspace['AZ'])
    result.append(eng.workspace['wx'])
    result.append(eng.workspace['wy'])
    result.append(eng.workspace['wz'])
    result.append(eng.workspace['actPower'])
    result.append(eng.workspace['R_ankle_roll_angle'])
    result.append(eng.workspace['R_ankle_roll_speed'])
    result.append(eng.workspace['R_ankle_roll_torque'])
    result.append(eng.workspace['R_ankle_pitch_angle'])
    result.append(eng.workspace['R_ankle_pitch_speed'])
    result.append(eng.workspace['R_ankle_pitch_torque'])
    result.append(eng.workspace['R_knee_angle'])
    result.append(eng.workspace['R_knee_speed'])
    result.append(eng.workspace['R_knee_torque'])
    result.append(eng.workspace['R_hip_roll_angle'])
    result.append(eng.workspace['R_hip_roll_speed'])
    result.append(eng.workspace['R_hip_roll_torque'])
    result.append(eng.workspace['R_hip_pitch_angle'])
    result.append(eng.workspace['R_hip_pitch_speed'])
    result.append(eng.workspace['R_hip_pitch_torque'])
    result.append(eng.workspace['L_ankle_roll_angle'])
    result.append(eng.workspace['L_ankle_roll_speed'])
    result.append(eng.workspace['L_ankle_roll_torque'])
    result.append(eng.workspace['L_ankle_pitch_angle'])
    result.append(eng.workspace['L_ankle_pitch_speed'])
    result.append(eng.workspace['L_ankle_pitch_torque'])
    result.append(eng.workspace['L_knee_angle'])
    result.append(eng.workspace['L_knee_speed'])
    result.append(eng.workspace['L_knee_torque'])
    result.append(eng.workspace['L_hip_roll_angle'])
    result.append(eng.workspace['L_hip_roll_speed'])
    result.append(eng.workspace['L_hip_roll_torque'])
    result.append(eng.workspace['L_hip_pitch_angle'])
    result.append(eng.workspace['L_hip_pitch_speed'])
    result.append(eng.workspace['L_hip_pitch_torque'])

    #sensor_res=eng.workspace['yaw']#고치기
    #reward_result = eng.workspace['simout1']
    #print(result)
    # 필요한 데이터를 추출하여 numpy 배열로 변환
    #state = np.array(result)
    
    return result

simulation & save

In [3]:
feature_names=["cumulative_reward", "yaw", "pitch", "roll", "X", "Y", "Z", "VX", "VY", "VZ",
    "AX", "AY", "AZ", "wx", "wv", "wz", "actPower", "R_ankle_roll_angle", 'R_ankle_roll_speed', 
    'R_ankle_roll_torque', 'R_ankle_pitch_angle', 'R_ankle_pitch_speed', 'R_ankle_pitch_torque',
    'R_knee_angle', 'R_knee_speed', 'R_knee_torque', 'R_hip_roll_angle', 'R_hip_roll_speed',
    'R_hip_roll_torque', 'R_hip_pitch_angle', 'R_hip_pitch_speed', 'R_hip_pitch_torque',
    'L_ankle_roll_angle','L_ankle_roll_speed', 'L_ankle_roll_torque', 'L_ankle_pitch_angle',
    'L_ankle_pitch_speed','L_ankle_pitch_torque','L_knee_angle','L_knee_speed','L_knee_torque',
    'L_hip_roll_angle', 'L_hip_roll_speed', 'L_hip_roll_torque','L_hip_pitch_angle','L_hip_pitch_speed', 'L_hip_pitch_torque']

In [4]:
# 새로운 엑셀 워크북 생성
wb = Workbook()

# 기본 시트 이름 변경
ws = wb.active
ws.title = feature_names[0]

# 추가 시트 생성
sheets = feature_names[1:]
for sheet_name in sheets:
    wb.create_sheet(title=sheet_name)

noiseStd=0.01
#agent_name="./models/DDPGtrainedAgent_3D_06_22_2024_1810.mat"
agent_name="./models/TD3trainedAgent_06_26_2024_1840.mat"
sim_count=100
for sim_num in range(1,sim_count+1):
    # 초기 동작 예시
    action = [0, 0]
    # 시뮬레이션 실행
    result = run_simulink_model(agent_name, noiseStd, action)

    # 각 시트에 데이터 추가
    for feature_num in range(0,len(feature_names)):
        ws = wb[feature_names[feature_num]] #select sheet

        data=np.array(result[feature_num]['signals']['values']) #get data
        flat_data = [item for sublist in data for item in sublist] #change to flat data list

        for i, value in enumerate(flat_data, start=1):
            ws.cell(row=i, column=sim_num, value=value)


# 엑셀 파일 저장
wb.save("./excel_files/"+agent_name.split(".")[1].split("/")[2]+"_"+str(noiseStd)+"_"+datetime.datetime.now().strftime('%Y_%m_%d_%H%M')+".xlsx")